In [2]:
from sklearn.model_selection import cross_val_score, train_test_split
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
import pandas as pd

In [3]:
df = pd.read_csv('./data/dataframefinal.csv',sep=',')
df = df.drop(columns=['order_id'])
df_test = pd.read_csv('./data/test_dataframe.csv',sep=',')
df_orders = pd.read_csv('./data/orders.csv',sep=';')
df = df.drop(columns=['distance'])
df = pd.get_dummies(df)

In [4]:
df.fillna(-1)
concat_col = df.select_dtypes(['object']).columns
df[concat_col] = df[concat_col].apply(lambda x:pd.factorize(x)[0])

In [5]:
df.head()

,product_id,units,late_order,weight,material_handling,weight_class,origin_port_Athens,origin_port_Barcelona,origin_port_Rotterdam,3pl_v_001,...,customer_Munich,customer_Naples,customer_Paris,customer_Porto,customer_Prague,customer_Rome,customer_Stockholm,customer_Turin,customer_Valencia,customer_Vienna
0,1692723,9,True,1778,5,8,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1692723,0,False,1778,5,8,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1692723,4,False,1778,5,8,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1692723,1,False,1778,5,8,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1692723,7,False,1778,5,8,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df.loc[:,df.columns != 'late_order']
y = df['late_order']

In [7]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix


In [8]:
over = SMOTE()
X, y = over.fit_resample(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X),len(y))
from collections import Counter
counter = Counter(y)
estimate = float(counter[0]/counter[1])

174070 174070


In [ ]:
xgb = XGBClassifier(scale_pos_weight=estimate)

In [ ]:
xgb.fit(X_train, y_train)

XGBClassifier(scale_pos_weight=1.0)

In [ ]:
y_train_hat = xgb.predict(X_train)
y_train_hat_probs = xgb.predict_proba(X_train)[:,1]

train_accuracy = accuracy_score(y_train, y_train_hat)*100
train_auc_roc = roc_auc_score(y_train, y_train_hat_probs)*100

print('Training AUC: %.4f %%' % train_auc_roc)

print('Training accuracy: %.4f %%' % train_accuracy)

Training AUC: 92.6887 %
Training accuracy: 85.4491 %


In [ ]:
y_test_hat = xgb.predict(X_test)
y_test_hat_probs = xgb.predict_proba(X_test)[:,1]

test_accuracy = accuracy_score(y_test, y_test_hat)*100
test_auc_roc = roc_auc_score(y_test, y_test_hat_probs)*100

print('Training AUC: %.4f %%' % train_auc_roc)

print('Training accuracy: %.4f %%' % train_accuracy)

Training AUC: 92.6887 %
Training accuracy: 85.4491 %


In [ ]:
df_test.head()

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,weight,material_handling,weight_class,distance
0,0e364fa5c795,Rotterdam,v_002,CRF,Warsaw,Munich,1687644,4,1974.0,5,8,1951.6946
1,6c8c11251114,Rotterdam,v_002,CRF,Venlo,Barcelona,1687644,1,1974.0,5,8,1283.4637
2,81d3be5e1eff,Athens,v_002,DTD,Warsaw,Lyon,1687644,8,1974.0,5,8,2982.0120
3,870a9b65a1b5,Rotterdam,v_001,CRF,Dusseldorf,Berlin,1687644,8,1974.0,5,8,654.6792
4,eff072956d66,Rotterdam,v_002,CRF,Rome,Milan,1687644,0,1974.0,5,8,1746.5587


In [ ]:
df_test = df_test.drop(columns=['distance'])
df_test = pd.get_dummies(df_test)
pred_proba = xgb.predict_proba(df_test)
submission = pd.DataFrame({'order_id':df_test.order_id, 'late_order':pred_proba[:,1]})
submission.to_csv('submission_kaggle_9.csv')